In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.0 MB/s eta 0:00:00


In [2]:
import sys
!{sys.executable} -m pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.1 MB/s eta 0:00:00


In [3]:
import sys
!{sys.executable} -m pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=84be7cb5308127e2246deaa845cd7efa34d11b8377188b7bb5aba9ab9741bedd
  Stored in directory: /home/idies/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.29
    Uninstalling GitPython-3.1.29:
      Successfully uninstalled GitPython-3.1.29


In [4]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [5]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='HistGradientBoosting Classifier',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /home/idies/workspace/Temporary/s.hossain18/scratch/jobs/20230805/20230805235733-245584/wandb/run-20230806_022252-fyyijf76
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run HistGradientBoosting Classifier
wandb: ⭐️ View project at https://wandb.ai/sakhawat/exoplanetml
wandb: 🚀 View run at https://wandb.ai/sakhawat/exoplanetml/runs/fyyijf76


In [6]:
import numpy as np

In [7]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [8]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [9]:
from skopt import BayesSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier

In [10]:
# Create an instance of the classifier
hist_gb_classifier = HistGradientBoostingClassifier(max_leaf_nodes=None,random_state=43)


In [11]:
# Define the parameter grid for HistGradientBoostingClassifier
params = {
    'max_iter': (100, 150),
    'min_samples_leaf': (5, 10),
    'l2_regularization': (1e-10, 1),
    'learning_rate': (0.01, 0.1)
}

In [12]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [13]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(hist_gb_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [14]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=41),
              estimator=HistGradientBoostingClassifier(max_leaf_nodes=None,
                                                       random_state=43),
              n_iter=20, n_jobs=-1,
              search_spaces={'l2_regularization': (1e-10, 1),
                             'learning_rate': (0.01, 0.1),
                             'max_iter': (100, 150),
                             'min_samples_leaf': (5, 10)})

In [15]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

Train score: 0.8083461226564675
Test score: 0.8253323485967504
Best hyperparameters: OrderedDict([('l2_regularization', 0.19429889580594611), ('learning_rate', 0.0893435789150355), ('max_iter', 139), ('min_samples_leaf', 10)])


In [16]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [17]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  Test score ▁
wandb: Train score ▁
wandb: 
wandb: Run summary:
wandb:  Test score 0.82533
wandb: Train score 0.80835
wandb: 
wandb: 🚀 View run HistGradientBoosting Classifier at: https://wandb.ai/sakhawat/exoplanetml/runs/fyyijf76
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230806_022252-fyyijf76/logs
